In [ ]:
from googleAPI import *
from pymongo import MongoClient
from crawling import ProgressBar
from youtubesearchpython import *
from pprint import pprint

In [ ]:
with open("mongodb.txt") as f:
    mongodb_uri = f.readline().strip()
client = MongoClient(mongodb_uri)
db = client["youtube"]
searchData = db["searchData"]

In [ ]:
videosSearch = VideosSearch('')
pprint(videosSearch.result())

In [ ]:
suggestionData = db["suggestionData"]
suggestions = Suggestions(language = 'kr', region = 'KR')

In [ ]:
import re

hangul = re.compile("[^가-힣]+")

In [ ]:
total_data = set()
# for i in range(44032, 55216):
start, end = 54000, 55216
error_count = 0
bar = ProgressBar(end-start)
for i in range(start, end):
    bar.next()
    total_data.add(chr(i))
    try:
        data = suggestions.get(chr(i), mode = ResultMode.dict)
        for j in data["result"]:
            if len(hangul.findall(j)) == 0:
                total_data.add(j)
    except:
        error_count += 1
total_data = list(total_data)
total_data.sort()
print(error_count, total_data)

In [ ]:
suggestionData.insert_one({"result":total_data})

In [ ]:
all_data = list(suggestionData.find())
finalSuggestionData = db["finalSuggestionData"]
total_search_data = set()
for data in all_data:
    for keyword in data["result"]:
        total_search_data.add(keyword)
total_search_data = list(total_search_data)
total_search_data.sort()
print(len(total_search_data))

In [ ]:
finalSuggestionData.drop()

In [ ]:
divide = 50
size = len(total_search_data)//divide+1
for i in range(divide):
    finalSuggestionData.insert_one({"id":i, "result":total_search_data[size*i:min(size*(i+1), len(total_search_data))]})

In [ ]:
Video.get("MhQKe-aERsU")

In [ ]:
for data_num in range(9,13):
    # data_num = 8
    total = finalSuggestionData.find_one({"id":data_num})["result"]
    id_collection = set()
    bar = ProgressBar(len(total))
    error_count = 0
    for target in total:
        bar.next()
        customSearch = CustomSearch(target, "CAMSAhAB", language="kr", region="KR")
        while(True):
            search_result = customSearch.result()["result"]
            if len(search_result) == 0:
                break
            for result in search_result:
                try:
                    viewCount = int(result["viewCount"]["text"].split(" views")[0].replace(",",""))
                    if viewCount < 10**6:
                        break
                    id_collection.add(result["id"])
                except:
                    continue
            else:
                customSearch.next()
                continue
            break
    print(error_count, len(id_collection))
    videoIdData = db["videoIdData"]
    videoIdData.insert_one({"result":list(id_collection)})


In [ ]:
print(len(set(id_collection)))

In [ ]:
customSearch12 = CustomSearch("애플", "CAMSAhAB", language="kr", region="KR") # preprocess english or not?
customSearch12.result()

In [ ]:
searchData.find_one()